# Загрузка датасета VoxTube

Задаём директорию сохранения на диске

In [1]:
import os

In [2]:
SAVE_PATH = '../dataset/'

Сперва проверим директорию `SAVE_PATH`. Так как датасет большой, его загрузка может осуществляться в несколько подходов, поэтому полезно вначале проверить, сколько уже удалось загрузить.

In [3]:
from collections import defaultdict

speakers = defaultdict(int) # количество загруженных аудио для каждого спикера

if os.path.exists(SAVE_PATH):
  spk_dirs = os.listdir(SAVE_PATH)
  for spk_dir in spk_dirs:
    speakers[spk_dir] = len(os.listdir(SAVE_PATH + spk_dir + '/'))

else:
  os.makedirs(SAVE_PATH)

In [4]:
speakers

defaultdict(int, {})

#### Авторизация на huggingface

Сначала необходимо перейти по ссылке: https://huggingface.co/datasets/voice-is-cool/voxtube и принять соглашение.

После этого нужно сгенерировать личный токен.

In [ ]:
TOKEN = "<YOUR TOKEN HERE>"

os.environ["HF_TOKEN"] = TOKEN

In [ ]:
# или используйте hugginface консоль для авторизации
#!hf auth login <token>

### Класс обработчик датасета

In [ ]:
from datasets import load_dataset
import soundfile as sf
from tqdm.auto import tqdm


class VoxtubeLoader:
  voxtube_len = 4_459_754

  def __init__(self, log_step: int = 1000,
               save_dir: str = SAVE_PATH,
               log_file: str = './log.txt'):
    self.log_step = log_step # частота логирования текущей записи
    self.max_spk_audios = 25 # максимальное количество аудио, которые загружаются на 1-го спикера
    self.save_dir = save_dir
    self.log_file = log_file
    self.samples_processed = 0 # счётчик обработанных записей датасета VoxTube
    self.speakers = defaultdict(int) # встреченные спикеры и количество аудио для них


  def log_info(self, log: str):
    """Логгирование в файл нового сообщения с новой строки"""
    with open(self.log_file, 'a') as log_file:
      log_file.write(log)


  def init_state(self, samples_processed: int, speakers: defaultdict):
    """Если запускаем обработку датасета не с нуля"""
    self.samples_processed = samples_processed
    self.speakers = speakers


  def save_audio(self, audio_decoder, audio_path: str):
    samples = audio_decoder.get_all_samples()

    wav = samples.data[0].numpy()
    sr = samples.sample_rate

    sf.write(audio_path, wav, sr)


  def process_sample(self, spk, audio_decoder, language):
    if language == 'russian':
      if spk in self.speakers:
        if self.speakers[spk] < self.max_spk_audios:
          # добавляем в директорию этого спикера новое аудио
          audio_path = self.save_dir + f"{spk}/" + f"{self.speakers[spk] + 1}.wav"
          self.save_audio(audio_decoder, audio_path)
          self.speakers[spk] += 1
      else:
        self.log_info(f"+ Найден новый спикер: {spk}\n")
        # добавляем новую директорию для этого спикера и грузим аудио туда
        os.makedirs(self.save_dir + f"{spk}/")
        audio_path = self.save_dir + f"{spk}/" + f"{1}.wav"
        self.save_audio(audio_decoder, audio_path)
        self.speakers[spk] += 1


  def load(self):
    self.dataset = load_dataset("voice-is-cool/voxtube", split="train", streaming=True)

    self.dataset = self.dataset.skip(self.samples_processed)

    self.log_info(f"Начало обработки датасета.\n\n")

    try:
      for sample in self.dataset:
        spk, audio_decoder, language = sample['spk_id'], sample['audio'], sample['language']
        self.process_sample(spk, audio_decoder, language)

        self.samples_processed += 1

        if self.samples_processed % self.log_step == 0:
          self.log_info(f"  $ Обработано: {self.samples_processed}$\n")

    except Exception as err:
      self.log_info(f":( Обработка датасета была прервана исключением: {err}")

    else:
      self.log_info(f":) Обработка датасета была успешно завершена.")

c:\Users\maksi\miniconda3\envs\speaker-embeddings-benchmark\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Загрузка датасета

In [ ]:
dataloader = VoxtubeLoader(save_dir=SAVE_PATH, log_file='../log.txt')

dataloader.init_state(samples_processed=0, speakers=speakers)
dataloader.load()